In [ ]:
pip install emoji

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 433.8/433.8 kB 3.2 MB/s eta 0:00:00


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
pip install pyvi

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.5/8.5 MB 25.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 57.8 MB/s eta 0:00:00


In [ ]:
import re
import emoji
import pandas as pd
from pyvi import ViTokenizer

In [ ]:
def remove_special_text(tweet):
  #Loại bỏ các kí tự và đường dẫn
  tweet = re.sub(r'(http\S+)|(@\S+)|RT|\#|!|:|\.|,', ' ', tweet)
  # Chuyển emoji sang text
  tweet = emoji.demojize(tweet)
  #Thêm dấu khoảng trắng trước và sau dấu text emoji
  tweet = re.sub(r'(:[a-zA-Z0-9_-]+:)', r' \1 ', tweet)

  tweet = tweet.replace('-', '_')
  tweet = tweet.replace(':', '_')

  characters_to_remove = [' ̉ ', ' ̃ ', ' ̣ ', ' ́ ', ' ̀ ']
  for char in characters_to_remove:
    tweet = tweet.replace(char.strip(), '')
  return tweet


def correct_Word(tweet):
    # Chuẩn hóa từ theo tự điển
    errorWord_dict = pd.read_csv('/content/drive/MyDrive/KLTN/dictionary/error_word.csv')
    for index, row in errorWord_dict.iterrows():
        if row['incorrect_Word'] in tweet:
            tweet  = tweet.replace(row['incorrect_Word'], row['correct_Word'])
    return tweet


def tokenization(tweet):
  token_dict = pd.read_csv('/content/drive/MyDrive/KLTN/dictionary/token_word.csv')
  # Duyệt qua từng dòng trong từ điển token
  for index, row in token_dict.iterrows():
    # Nếu từ trong từ điển được tìm thấy trong tweet
    if row['Word'] in tweet:
    # Thay thế từ bằng token tương ứng
      tweet = tweet.replace(row['Word'], row['Token'])

    # Thay thế các dấu gạch dưới kép bằng dấu gạch dưới duy nhất
    #tweet = tweet.replace('__', ' _')

  return tweet

def remove_stopWord(tweet):
    # Chuẩn hóa từ theo từ điển
    stopWord_dict = pd.read_csv('/content/drive/MyDrive/KLTN/dictionary/stop_word.csv')
    for index, row in stopWord_dict.iterrows():
        stop_word = row['stop_Word']
        # Loại bỏ stop word nếu nó xuất hiện trong tweet
        tweet = ' '.join([word for word in tweet.split() if word != stop_word])
    return tweet


def pre_process(tweet):
    tweet = remove_special_text(tweet)
    tweet = correct_Word(tweet)
    tweet = tokenization(tweet)
    tweet = remove_stopWord(tweet)
    tokens = ViTokenizer.tokenize(tweet)
    tokens = tokens.split()
    return tokens

In [ ]:
def process_dataset(input_file, output_file):
    try:
        data = pd.read_csv(input_file, encoding='utf-8')
    except FileNotFoundError:
        print("File input không tồn tại!")
        return

    print("Input length:", len(data))
    rows = []

    for index, row in data.iterrows():
        tweet = row['Reviews']
        tokens = pre_process(tweet)
        sentiment = row['Sentiment']
        rows.append([sentiment, tokens, tweet])

    df = pd.DataFrame(rows, columns=['Sentiment', 'Tokens', 'Reviews'])

    df.to_csv(output_file, index=False, encoding='utf-8')


process_dataset('/content/drive/MyDrive/KLTN/input/UIT.csv', '/content/drive/MyDrive/KLTN/output/UIT_Final.csv')

Input length: 14876
